In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import img_to_array

# Function to create a CNN model for CAPTCHA solving
def create_captcha_solver():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 200, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(36, activation='softmax')  # Assuming 36 possible characters (A-Z, 0-9)
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Example usage: Create the model
model = create_captcha_solver()

# Train the model on CAPTCHA data (not shown here - you need labeled CAPTCHA dataset)
# model.fit(x_train, y_train, epochs=10, batch_size=32)

# Save the model
model.save('captcha_solver.h5')

In [ ]:
def preprocess_captcha_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to binarize the image
    _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Resize image to fit model input shape
    resized_image = cv2.resize(thresholded, (200, 50))  # Adjust dimensions based on your model
    # Normalize the image
    resized_image = resized_image.astype('float32') / 255.0
    # Add an extra dimension for channel (as the model expects a 4D input)
    resized_image = np.expand_dims(resized_image, axis=-1)
    return resized_image

# Example usage: Preprocess an image
preprocessed_image = preprocess_captcha_image('captcha_image.png')

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import random
import string

# Function to generate a random CAPTCHA image
def generate_captcha_image(text, width=200, height=50):
    # Create a new image with white background
    image = Image.new('RGB', (width, height), color=(255, 255, 255))
    draw = ImageDraw.Draw(image)
    
    # Load a font (use a path to a .ttf file for a custom font)
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 36)
    
    # Add random text
    text = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    text_width, text_height = draw.textsize(text, font=font)
    
    # Position the text at the center
    position = ((width - text_width) / 2, (height - text_height) / 2)
    draw.text(position, text, fill=(0, 0, 0), font=font)
    
    # Add some noise (lines, dots, etc.) to make the CAPTCHA harder to solve
    for _ in range(random.randint(5, 15)):
        x1, y1 = random.randint(0, width), random.randint(0, height)
        x2, y2 = random.randint(0, width), random.randint(0, height)
        draw.line([x1, y1, x2, y2], fill=(0, 0, 0), width=random.randint(1, 3))
    
    # Save the image
    image.save('captcha_image.png')
    return text

# Example usage: Generate a CAPTCHA image
captcha_text = generate_captcha_image("RandomCaptcha")
print("Generated CAPTCHA Text: ", captcha_text)

In [ ]:
def predict_captcha(image_path, model):
    preprocessed_image = preprocess_captcha_image(image_path)
    prediction = model.predict(np.expand_dims(preprocessed_image, axis=0))  # Batch dimension
    predicted_class = np.argmax(prediction, axis=1)
    # Map predicted class to characters (use a character mapping dictionary)
    characters = string.ascii_uppercase + string.digits
    predicted_text = ''.join([characters[i] for i in predicted_class])
    return predicted_text

# Example usage: Predict CAPTCHA text
predicted_text = predict_captcha('captcha_image.png', model)
print(f"Predicted CAPTCHA Text: {predicted_text}")